In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
train_data.head()

In [ ]:
test_data=pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
sns.set(style="whitegrid")
sns.stripplot(x='type',y='bone_length',data=train_data)

In [ ]:
sns.set(style="whitegrid")
sns.stripplot(x='type',y='rotting_flesh',data=train_data)

In [ ]:
sns.stripplot(x='type',y='color',data=train_data)

In [ ]:
train_data.color.value_counts()

In [ ]:
test_data.color.value_counts()

In [ ]:
train_data.bone_length.plot(kind='hist')

In [ ]:
test_data.bone_length.plot(kind='hist')

In [ ]:
train_data=pd.concat([train_data,pd.get_dummies(train_data.color)],axis=1)
train_data=train_data.drop('color',axis=1)
train_data.head()

In [ ]:
test_data=pd.concat([test_data,pd.get_dummies(test_data.color)],axis=1)
test_data=test_data.drop('color',axis=1)
test_data.head()

In [ ]:
X=train_data.drop(['id','type'],axis=1)
y=train_data.type

In [ ]:
Y=pd.get_dummies(y)
Y

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,random_state=70)

In [ ]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
X_train

In [ ]:
from tensorflow import keras

In [ ]:
from keras.layers import Dense,Activation
from keras.models import Sequential
model=Sequential()

model.add(Dense(20,input_shape=(X.shape[1],)))
model.add(Activation('relu'))

model.add(Dense(20))
model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
trained_model=model.fit(X_train,Y_train,
         batch_size=16,
         epochs=20,
         verbose=2,
         validation_data=(X_test,Y_test))

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(trained_model.history['accuracy'],'r',label='Training accuracy')
plt.plot(trained_model.history['val_accuracy'],'b',label='Validation accuracy')
plt.legend()
plt.show()

In [ ]:
pred=model.predict(test_data.drop('id',axis=1))

In [ ]:
pred

In [ ]:
pred_final=np.argmax(pred,axis=1)
pred_final

In [ ]:
Submission = pd.DataFrame({'id':test_data['id'], 'type':pred_final})
Submission.set_index('id',inplace=True)
Submission.head()

In [ ]:
type_map={0:'Ghost' ,1:'Ghoul' ,2:'Goblin'}
Submission.type=Submission.type.map(type_map)
Submission.head()

In [ ]:
Submission.to_csv('Submission.csv')
